## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Anadyr,RU,64.7500,177.4833,52.30,50,20,29.08,few clouds
1,1,Nuuk,GL,64.1835,-51.7216,50.61,62,75,4.61,broken clouds
2,2,Chokurdakh,RU,70.6333,147.9167,43.07,67,80,13.11,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,50.29,86,0,8.68,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,44.47,97,100,1.03,overcast clouds
5,5,Tamazunchale,MX,21.2667,-98.7833,88.77,47,100,4.68,overcast clouds
6,6,Otacilio Costa,BR,-27.4831,-50.1219,50.07,98,100,2.13,overcast clouds
7,7,Butaritari,KI,3.0707,172.7902,82.31,77,10,19.01,light rain
8,8,Eyl,SO,7.9803,49.8164,77.20,67,98,27.25,overcast clouds
9,9,Lobito,AO,-12.3481,13.5456,69.10,84,70,4.74,broken clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 25
What is your desired maximum temperature for your trip? 95


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Anadyr,RU,64.7500,177.4833,52.30,50,20,29.08,few clouds
1,1,Nuuk,GL,64.1835,-51.7216,50.61,62,75,4.61,broken clouds
2,2,Chokurdakh,RU,70.6333,147.9167,43.07,67,80,13.11,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,50.29,86,0,8.68,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,44.47,97,100,1.03,overcast clouds
5,5,Tamazunchale,MX,21.2667,-98.7833,88.77,47,100,4.68,overcast clouds
6,6,Otacilio Costa,BR,-27.4831,-50.1219,50.07,98,100,2.13,overcast clouds
7,7,Butaritari,KI,3.0707,172.7902,82.31,77,10,19.01,light rain
8,8,Eyl,SO,7.9803,49.8164,77.20,67,98,27.25,overcast clouds
9,9,Lobito,AO,-12.3481,13.5456,69.10,84,70,4.74,broken clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Anadyr,RU,64.7500,177.4833,52.30,50,20,29.08,few clouds
1,1,Nuuk,GL,64.1835,-51.7216,50.61,62,75,4.61,broken clouds
2,2,Chokurdakh,RU,70.6333,147.9167,43.07,67,80,13.11,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,50.29,86,0,8.68,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,44.47,97,100,1.03,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
686,686,Pljevlja,ME,43.3333,19.3333,53.33,56,53,1.68,broken clouds
687,687,Sorland,NO,67.6670,12.6934,51.66,84,88,13.00,overcast clouds
688,688,Qaqortoq,GL,60.7167,-46.0333,47.53,78,100,3.96,overcast clouds
689,689,Crowley,US,30.2141,-92.3746,92.98,66,86,9.01,overcast clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Anadyr,RU,52.30,few clouds,64.7500,177.4833,
1,Nuuk,GL,50.61,broken clouds,64.1835,-51.7216,
2,Chokurdakh,RU,43.07,broken clouds,70.6333,147.9167,
3,Busselton,AU,50.29,clear sky,-33.6500,115.3333,
4,Mataura,NZ,44.47,overcast clouds,-46.1927,168.8643,
5,Tamazunchale,MX,88.77,overcast clouds,21.2667,-98.7833,
6,Otacilio Costa,BR,50.07,overcast clouds,-27.4831,-50.1219,
7,Butaritari,KI,82.31,light rain,3.0707,172.7902,
8,Eyl,SO,77.20,overcast clouds,7.9803,49.8164,
9,Lobito,AO,69.10,broken clouds,-12.3481,13.5456,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Sk

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Anadyr,RU,52.30,few clouds,64.7500,177.4833,Отке17.ру — жильё для командированных
1,Nuuk,GL,50.61,broken clouds,64.1835,-51.7216,Vandrehuset
3,Busselton,AU,50.29,clear sky,-33.6500,115.3333,Observatory Guest House
4,Mataura,NZ,44.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
5,Tamazunchale,MX,88.77,overcast clouds,21.2667,-98.7833,Hotel Paraiso Huasteco


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))